CAN I SAVE THE CHANGES???

In [ ]:
!pip install uniparser_morph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.9 MB/s eta 0:00:00


In [1]:
!git clone https://github.com/DionysiusSidorius/project-753.git

Cloning into 'project-753'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 98 (delta 35), reused 31 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (98/98), 7.97 MiB | 3.29 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [ ]:
from uniparser_morph import Analyzer
a = Analyzer()
# a.lexFile = "/content/project-753/lexemes.txt"
a.paradigmFile = "/content/project-753/paradigms.txt"
a.load_grammar()

fatal: destination path 'project-753' already exists and is not an empty directory.


ModuleNotFoundError: No module named 'uniparser_morph'

In [ ]:
analyses = a.analyze_words(['test', "buenos"])
print(analyses)

[[<Wordform object>
test
test; NOUN,sg
test
STEM
trans_en	testus
], [<Wordform object>
buenos
bueno; ADJ,M.Pl
buen-os
STEM
trans_en	good
]]


In [ ]:
with open('lexemes.txt', 'w') as lex:
  lex.write(
      """-lexeme
 lex: test
 stem: test.
 gramm: NOUN
 paradigm: NOM
 trans_en: testus
      """
  )
with open('paradigms.txt', 'w') as par:
  par.write(
      """-paradigm: NOM
 -flex: .
  gramm: sg
 -flex: .s
  gramm: pl
  gloss: PL
 -flex: .'s
  gramm: sg,poss
  gloss: POSS
 -flex: .s'
  gramm: pl,poss
  gloss: POSS.PL
      """
  )


In [3]:
import pandas as pd
df = pd.read_csv('/content/project-753/spanish.csv', sep=None)

<ipython-input-3-d79e48a46742>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('/content/project-753/spanish.csv', sep=None)


In [ ]:
df.head()

,IDX,WORD,POS,MEANINGS,EXAMPLES,ADDITIONAL_INFO
0,e4348,a,sust.,"{'1': 'Letra del abecedario.', '2': 'Usado par...","{'1': ['LAP (1250?-1279?) fol. 113v87, CCN: 00...","{'Formas atestiguadas:\xa0': ['a'], 'Véase tam..."
1,e4349,a,prep.,{'1': 'Denota el complemento de la acción del ...,"{'1': ['LAP (1250?-1279?) fol. 1r33, CCN: 0000...","{'Formas atestiguadas:\xa0': ['a', 'ad', 'ha']}"
2,e132,aabruz,extranj.,{'1': 'Calcedonia. '},"{'1': ['LAP (1250?-1279?) fol. 118r41, CCN: 00...","{'Formas atestiguadas:\xa0': ['aabruz', 'aambr..."
3,e137,aanca,sust.,"{'1': 'Pieza en el juego de ajedrez indio.', '...","{'1': ['ACE (1283) fol. 82r5, CCN: 0000028, el...",{'Formas atestiguadas:\xa0': ['aanca']}
4,e138,aaptamiento,sust.,"{'1': 'Aptitud, cualidad que hace que un objet...","{'1': ['GE1 (1272-1275) fol. 169v10, CCN: 0000...",{'Formas atestiguadas:\xa0': ['a abtamiento']}


In [48]:
with open('lexemes.txt', 'w') as lex:
  pass

###nomina

In [ ]:
#NOUN(_V)
import re
dict_noun_V = df[
    (df['WORD'].str.strip().str.fullmatch(r".+[aoeiuáóéíú]\b")) & (df['POS'].str.strip() == 'sust.')
]


with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_noun_V['WORD'], dict_noun_V['ADDITIONAL_INFO']):
    forms = set([re.sub(r'(.+)s', r'\1', form) for form in set(eval(other)['Formas atestiguadas:\xa0'])])
    forms.discard(word)
    lex.write(
f"""-lexeme
 lex: {word}
 stem: {word}.{'|'+'|'.join([form+"." for form in forms if form.count(' ') == 0]) if forms else ''}
 gramm: NOUN
 paradigm: NOM
"""
    )

In [ ]:
#NOUN(_C)
import re
dict_noun_C = df[
    (df['WORD'].str.strip().str.fullmatch(r'.*(?!.[aoeiuáóéíú]|os).{2}')) & (df['POS'].str.strip() == 'sust.')
]


with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_noun_C['WORD'], dict_noun_C['ADDITIONAL_INFO']):
    forms = set([re.sub(r'(.+)es', r'\1', form) if word[-2:]!='es' else re.sub(r'(.+ess?)es', r'\1', form) for form in set(eval(other)['Formas atestiguadas:\xa0'])])
    forms.discard(word)
    lex.write(
f"""-lexeme
 lex: {word}
 stem: {word}.{'|'+'|'.join([form.strip()+"." for form in forms if form.strip().count(' ') == 0]) if forms else ''}
 gramm: NOUN
 paradigm: NOM_C
"""
    )

In [5]:
#NOUN(_os)
import re
dict_noun_os = df[
    (df['WORD'].str.strip().str.fullmatch(r'.+os\b')) & (df['POS'].str.strip() == 'sust.')
]


with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_noun_os['WORD'], dict_noun_os['ADDITIONAL_INFO']):
    forms = set(eval(other)['Formas atestiguadas:\xa0'])
    forms.discard(word)
    if any([form[-2:]=='es' for form in forms]):
      forms = [re.sub(r'(.+)es', r'\1', form) for form in forms]
      lex.write(
f"""-lexeme
 lex: {word}
 stem: {word}.{'|'+'|'.join([form.strip()+"." for form in forms if form.strip().count(' ') == 0]) if forms else ''}
 gramm: NOUN
 paradigm: NOM_C
""")
    else:
      lex.write(
f"""-lexeme
 lex: {word}
 stem: {word}.{'|'+'|'.join([form.strip()+"." for form in forms if form.strip().count(' ') == 0]) if forms else ''}
 gramm: NOUN
 paradigm: NOM_os
"""
    )

In [32]:
#ADJ(_V)
import re
dict_adj_V = df[
    (df['WORD'].str.strip().str.fullmatch(r".+o\b")) & (df['POS'].str.strip() == 'adj.')
]


with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_adj_V['WORD'], dict_adj_V['ADDITIONAL_INFO']):
    word = re.sub(r'(.+)(o)', r'\1', word)
    if not isinstance(other, str):
      forms = set()
    else:
      forms = set([re.sub(r'(.+)([oa]s?)', r'\1', form) for form in set(eval(other)['Formas atestiguadas:\xa0'])])
    forms.discard(word)
    lex.write(
f"""-lexeme
 lex: {word+'o'}
 stem: {word}.{'|'+'|'.join([form+"." for form in forms if form.count(' ') == 0]) if forms else ''}
 gramm: ADJ
 paradigm: ADJ
"""
    )

In [28]:
#ADJ(_e)
import re
dict_adj_e = df[
    (df['WORD'].str.strip().str.fullmatch(r".+e\b")) & (df['POS'].str.strip() == 'adj.')
]


with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_adj_e['WORD'], dict_adj_e['ADDITIONAL_INFO']):
    word = re.sub(r'(.+)(e)', r'\1', word)
    if not isinstance(other, str):
      forms = set()
    else:
      forms = set([re.sub(r'(.+)(es?)', r'\1', form) for form in set(eval(other)['Formas atestiguadas:\xa0'])])
    forms.discard(word)
    lex.write(
f"""-lexeme
 lex: {word+'e'}
 stem: {word}.{'|'+'|'.join([form+"." for form in forms if form.count(' ') == 0]) if forms else ''}
 gramm: ADJ
 paradigm: ADJ_e
"""
  )

In [38]:
#ADJ(_C/_Ca)
import re
dict_adj_C = df[
    (df['WORD'].str.strip().str.fullmatch(r'.*(?!.[aoeiuáóéíú]|os).{2}')) & (df['POS'].str.strip() == 'adj.')
]


with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_adj_C['WORD'], dict_adj_C['ADDITIONAL_INFO']):
    if not isinstance(other, str):
      forms = set()
    else:
      forms = set([re.sub(r'(.+)(es|(?<=a)s)\b', r'\1', form) for form in set(eval(other)['Formas atestiguadas:\xa0'])])
    if any([form[-1]=='a' for form in forms]):
      forms = set([re.sub(r'(.+)(a)\b', r'\1', form) for form in forms])
      forms.discard(word)
      lex.write(
  f"""-lexeme
  lex: {word}
  stem: {word}.{'|'+'|'.join([form+"." for form in forms if form.count(' ') == 0]) if forms else ''}
  gramm: ADJ
  paradigm: ADJ_Ca
  """
    )
    else:
      forms.discard(word)
      lex.write(
  f"""-lexeme
  lex: {word}
  stem: {word}.{'|'+'|'.join([form+"." for form in forms if form.count(' ') == 0]) if forms else ''}
  gramm: ADJ
  paradigm: ADJ_C
  """
    )

In [39]:
#ADJ(_os) - son numerales (ADJ_V) o simplemente adjetivos (ADJ_C(a))

###ál

In [41]:
#interj
import re
dict_noun_V = df[
    df['POS'].str.strip() == 'interj.'
]


with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_noun_V['WORD'], dict_noun_V['ADDITIONAL_INFO']):
    forms = set(eval(other)['Formas atestiguadas:\xa0'])
    forms.discard(word)
    lex.write(
f"""-lexeme
 lex: {word}
 stem: {word}.{'|'+'|'.join([form+"." for form in forms if form.count(' ') == 0]) if forms else ''}
 gramm: INTERJ
 paradigm: ZERO
"""
    )

In [43]:
#adv
import re
dict_noun_V = df[
    df['POS'].str.strip() == 'adv.'
]


with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_noun_V['WORD'], dict_noun_V['ADDITIONAL_INFO']):
    forms = set(eval(other)['Formas atestiguadas:\xa0'])
    forms.discard(word)
    lex.write(
f"""-lexeme
 lex: {word}
 stem: {word}.{'|'+'|'.join([form+"." for form in forms if form.count(' ') == 0]) if forms else ''}
 gramm: ADV
 paradigm: ZERO
"""
    )

In [45]:
#prep
import re
dict_noun_V = df[
    df['POS'].str.strip() == 'prep.'
]


with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_noun_V['WORD'], dict_noun_V['ADDITIONAL_INFO']):
    forms = set(eval(other)['Formas atestiguadas:\xa0'])
    forms.discard(word)
    lex.write(
f"""-lexeme
 lex: {word}
 stem: {word}.{'|'+'|'.join([form+"." for form in forms if form.count(' ') == 0]) if forms else ''}
 gramm: PREP
 paradigm: ZERO
"""
    )

In [49]:
#conj
import re
dict_noun_V = df[
    df['POS'].str.strip() == 'conj.'
]


with open('lexemes.txt', 'a') as lex:
  for word, other in zip(dict_noun_V['WORD'], dict_noun_V['ADDITIONAL_INFO']):
    forms = set(eval(other)['Formas atestiguadas:\xa0'])
    forms.discard(word)
    lex.write(
f"""-lexeme
 lex: {word}
 stem: {word}.{'|'+'|'.join([form+"." for form in forms if form.count(' ') == 0]) if forms else ''}
 gramm: PREP
 paradigm: ZERO
"""
    )

In [54]:
df['POS'].value_counts()

,count
POS,
sust.,5488
vrb.,1538
adj.,1245
Unidades pluriverbales:,830
extranj.,810
adv.,335
pron.,37
contracc.,19
prep.,16


In [52]:
dfi = df[df['POS'] == 'partíc. ']

In [53]:
dfi

,IDX,WORD,POS,MEANINGS,EXAMPLES,ADDITIONAL_INFO
988,e2747,apia,partíc.,Usado sólo como elemento de una locución.,"{'1': ['GE4 (1280) fol. 242v38, CCN: 0006428, ...","{'Formas atestiguadas:\xa0': ['appia'], 'Véase..."
3074,e8244,cuasimodo,partíc.,Usado como elemento de una unidad pluriverbal.,"{'1': ['GE1 (1272-1275) fol. 221v68, quasimodo...","{'Formas atestiguadas:\xa0': ['quasi modo', 'q..."
6883,e18701,moscada,partíc.,Usado sólo como elemento de una locución.,"{'1': ['PIC (1256?) fol. 20r26, CCN: 0045328, ...","{'Formas atestiguadas:\xa0': ['moscada', 'musc..."
7427,e20165,pardal,partíc.,Usado como elemento de una unidad pluriverbal.,"{'1': ['GE1 (1272-1275) fol. 327v85, CCN: 0048...","{'Formas atestiguadas:\xa0': ['pardal'], 'Véas..."
7530,e20457,pedrés,partíc.,Usado sólo como elemento de una unidad plurive...,"{'1': ['MOA (1250?-1300?) fol. 43v9, CCN: 0050...","{'Formas atestiguadas:\xa0': ['pedres'], 'Véas..."
8003,e21717,probática,partíc.,Piscina probática.,"{'7': ['GE4 (1280) fol. 135r83, CCN: 0053597, ...","{'Formas atestiguadas:\xa0': ['probatica'], 'V..."
8760,e23852,salmera,partíc.,Usado sólo como elemento de una unidad plurive...,"{'1': ['MOA (1250?-1300?) fol. 127v15, CCN: 00...","{'Formas atestiguadas:\xa0': ['salmar'], 'Véas..."
9286,e25393,sucotrino,partíc.,Perteneciente a la isla de Socotra.,"{'3': ['MOA (1250?-1300?) fol. 95r47, CCN: 006...","{'Formas atestiguadas:\xa0': ['çocotri', 'çoco..."
